In [1]:
import os

import imageio
import numpy as np
import matplotlib.pyplot as plt

import utoolbox.utils.files as fileutils
from utoolbox.container import Raster
from utoolbox.container.layouts import Volume

Retrieve file list from source directory.

In [2]:
source_folder = os.path.join(*["data", "20170112_RFiSHp2aLFCYC", "decon", "488"])
file_list = fileutils.list_files(
    source_folder,
    name_filters=[
        fileutils.ExtensionFilter('tif'),
        fileutils.SPIMFilter(channel=0)
    ]
)

print("[0] = {}".format(file_list[0]))

[0] = data/20170112_RFiSHp2aLFCYC/decon/488/cell4_ch0_stack0004_488nm_0040241msec_0007974972msecAbs_decon.tif


Load the image.

In [3]:
image = Raster(file_list[0], layout=Volume, spacing=(.100319, .102, .102))
print("layout={}".format(image.metadata.layout))

layout=<class 'utoolbox.container.layouts.Volume'>


In [4]:
image_sitk = image.to_sitk()

In [5]:
print(image_sitk)

Image (0x7fa530c790d0)
  RTTI typeinfo:   itk::Image<float, 3u>
  Reference Count: 1
  Modified Time: 779
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [810, 810, 140]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [810, 810, 140]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [810, 810, 140]
  Spacing: [0.102, 0.102, 0.100319]
  Origin: [0, 0, 0]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.102 0 0
0 0.102 0
0 0 0.100319

  PointToIndexMatrix: 
9.80392 0 0
0 9.80392 0
0 0 9.9682

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x7fa530c715c0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference Count: 1
  